# Определение возраста покупателей

## Описание проекта


Сетевой супермаркет «Хлеб-Соль» внедряет систему компьютерного зрения для обработки фотографий покупателей. Фотофиксация в прикассовой зоне поможет определять возраст клиентов, чтобы:

* анализировать покупки и предлагать товары, которые могут заинтересовать покупателей этой возрастной группы;
* контролировать добросовестность кассиров при продаже алкоголя.
Необходимо построить модель, которая по фотографии определит приблизительный возраст человека. В вашем распоряжении набор фотографий людей с указанием возраста.

Данные взяты с сайта ChaLearn Looking at People. Они находятся в папке /datasets/faces/.
В нашем распоряжении одна папка со всеми изображениями (/final_files) и csv-файл labels.csv с двумя колонками: file_name и real_age.

## Исследовательский анализ данных

Импортируем необходимые библиотеки и загрузим данные.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [ ]:
labels = pd.read_csv('/datasets/faces/labels.csv')
train_datagen = ImageDataGenerator(rescale=1./255)
train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory='/datasets/faces/final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        seed=12345)

Подробно рассмотрим данные.

In [ ]:
labels.info()

In [ ]:
labels


In [ ]:
labels.shape

In [ ]:
labels['real_age'].describe()

In [ ]:
labels[labels.duplicated()]

Посмотрим, как выглядят изображения в нашем наборе данных.

In [ ]:
x,y = train_gen_flow.next()
for i in range(0,15):
    image = x[i]
    plt.imshow(image)
    plt.title(f'Возраст - {y[i]}')
    plt.show()

In [ ]:
sns_plot = sns.distplot(labels['real_age'])
fig = sns_plot.get_figure();

#### Вывод:
Данные содержат два столбца: 
* фотографии людей различных возрастов 
* их реальный возраст

Всего в таблице 7591 запись, пропуски и дублликаты отсутствуют. Средний возраст почти совпадает с медианным - разница в 2 года.

Распределение немного смещено вправо, присутсвуют выбросы - к таким значениям относятся люди в возрасте за 70 лет, также видно, что очень много младенцев.

Подавляющее большинство фотографий в наборе данных - портрет крупным планом, но встречаются и в полный рост, где человек катается на качели или стоит боком. Некоторые изображения повернуты, поэтому при обучении модели будем применять аугментации - отражения по горизонтали. Есть черно-белые фото, фото без лица вообще, фото несоответствующее возрасту, фото с большим количеством шума, такие данные могут отрицательно сказаться на качестве обучения модели. Есть фото разных размеров, однако привыгрузке все они конвертируются к (224, 224).

## Обучение модели


(Код в этом разделе запускается в отдельном GPU-тренажёре, поэтому оформлен не как ячейка с кодом, а как код в текстовой ячейке)

def load_train(path):
    labels = pd.read_csv(path + 'labels.csv')
    train_datagen = ImageDataGenerator(validation_split=0.25, rescale=1 / 255, horizontal_flip=True)
    train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory=path + 'final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        subset = 'training',
        seed=12345
        )

    return train_gen_flow
 
def load_test(path):
    labels = pd.read_csv(path + 'labels.csv')
    validation_datagen = ImageDataGenerator(validation_split=0.25, rescale=1/255)
    test_gen_flow = validation_datagen.flow_from_dataframe(
        dataframe = labels,
        directory=path +'final_files/',
        x_col="file_name",
        y_col="real_age", 
        class_mode="raw", 
        target_size=(224,224), 
        batch_size=32,
        subset = "validation",
        seed=12345,
        )

    return test_gen_flow
  

 
def create_model(input_shape):
    backbone = ResNet50(input_shape=input_shape, weights='imagenet', include_top=False)
    model = Sequential()
    model.add(backbone)
    model.add(Dropout(0.2))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(1, activation='relu'))
    optimizer = Adam(lr=0.0003)
    model.compile(optimizer=optimizer, loss='mae', metrics=['mae'])

    return model
 
def train_model(model, train_data, test_data, batch_size=None, epochs=20,
                steps_per_epoch=None, validation_steps=None):
  
    model.fit(train_data, validation_data=test_data, batch_size=batch_size, 
              epochs=epochs, steps_per_epoch=steps_per_epoch, 
              validation_steps=validation_steps, verbose=2)

    return model

Train for 178 steps, validate for 60 steps
Epoch 1/20
2022-07-19 09:49:44.775656: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2022-07-19 09:49:45.093719: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
178/178 - 46s - loss: 11.1126 - mae: 11.1090 - val_loss: 16.9492 - val_mae: 16.9339
Epoch 2/20
178/178 - 38s - loss: 7.8144 - mae: 7.8148 - val_loss: 12.9293 - val_mae: 12.8796
Epoch 3/20
178/178 - 37s - loss: 6.5944 - mae: 6.5949 - val_loss: 14.7028 - val_mae: 14.6696
Epoch 4/20
178/178 - 37s - loss: 5.8665 - mae: 5.8667 - val_loss: 11.5766 - val_mae: 11.5183
Epoch 5/20
178/178 - 37s - loss: 5.3508 - mae: 5.3513 - val_loss: 7.4037 - val_mae: 7.3727
Epoch 6/20
178/178 - 37s - loss: 4.8245 - mae: 4.8248 - val_loss: 7.2007 - val_mae: 7.2247
Epoch 7/20
178/178 - 37s - loss: 4.4056 - mae: 4.4060 - val_loss: 7.5200 - val_mae: 7.5071
Epoch 8/20
Epoch 9/20
178/178 - 37s - loss: 4.0030 - mae: 4.0032 - val_loss: 7.2441 - val_mae: 7.2657
Epoch 10/20
178/178 - 37s - loss: 3.8519 - mae: 3.8521 - val_loss: 6.6965 - val_mae: 6.6879
178/178 - 37s - loss: 3.4495 - mae: 3.4496 - val_loss: 7.0280 - val_mae: 7.0523
Epoch 11/20
178/178 - 38s - loss: 3.3416 - mae: 3.3416 - val_loss: 7.8697 - val_mae: 7.8617
Epoch 12/20
178/178 - 37s - loss: 3.1959 - mae: 3.1961 - val_loss: 7.0813 - val_mae: 7.0992
Epoch 13/20
178/178 - 37s - loss: 2.9781 - mae: 2.9784 - val_loss: 6.4977 - val_mae: 6.5272
Epoch 14/20
178/178 - 37s - loss: 2.8346 - mae: 2.8347 - val_loss: 6.6604 - val_mae: 6.6585
Epoch 15/20
178/178 - 37s - loss: 2.7656 - mae: 2.7659 - val_loss: 7.0988 - val_mae: 7.1203
Epoch 16/20
178/178 - 37s - loss: 2.5620 - mae: 2.5622 - val_loss: 6.9002 - val_mae: 6.8813
Epoch 17/20
178/178 - 38s - loss: 2.5624 - mae: 2.5625 - val_loss: 6.2109 - val_mae: 6.2341
Epoch 18/20
178/178 - 37s - loss: 2.4270 - mae: 2.4268 - val_loss: 6.4669 - val_mae: 6.5021
Epoch 19/20
178/178 - 37s - loss: 2.4142 - mae: 2.4145 - val_loss: 6.2687 - val_mae: 6.2876
Epoch 20/20
178/178 - 37s - loss: 2.2845 - mae: 2.2844 - val_loss: 6.5072 - val_mae: 6.5113
WARNING:tensorflow:sample_weight modes were coerced from
  ...
    to  
  ['...']
60/60 - 9s - loss: 6.5072 - mae: 6.5113
Test MAE: 6.5113
```

## Анализ обученной модели

### Вывод:
Значение MAE ниже 8 достигнуто - 6.5113(с 20 эпохами), была создана нейронная сеть на основе архитектуры ResNet50, модель обучена на 75% данных и проверена на остальных. 

Результаты свидетельствуют,что модель с такой погрешностью не сможет помочь в определении возраста покупателя для продажи алкоголя. А вот с задачей "Анализировать покупки и предлагать товары, которые могут заинтересовать покупателей этой возрастной группы" модель вполне может справиться.

Дальнейшая тонкая настройка нейронной сети может дать лучшие результаты, также увеличение количества данных с их бОльшей корректностью могло бы улучшить результат.